In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import re
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup 

imdb = []

In [ ]:
def extract():
    # Genres of the movies for which the movies will be pulled
    # GENRES = ['action']
    GENRES = ['action', 'adventure', 'animation', 'biography', 'comedy', 'crime', 'war',
			  'drama', 'family', 'fantasy','history', 'horror', 
			  'music', 'musical', 'mystery', 'romance', 'sci-fi', 'sport', 'thriller','western']	

	# Upper Limit - How many movies should be pulled at max 
    upper_limit = 500000

    for genre in GENRES:
        movie_count_total = setMovieCountTotal(genre)

        # get number of pages, 50 items per page
        num_pages = int(movie_count_total / 50)
        print ('Genre:\t\t' + genre)
        print ('Num Pages:\t' + str(num_pages))
        print ()

        movie_index = 1

        for i in range(0, num_pages):
		
            url = getUrl(movie_index, genre)

            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')

            movies_article = soup.find(class_ = 'article')
            movies_list = movies_article.findAll('div', {'class': 'lister-item mode-advanced'})


            for movie in movies_list:
                stripInfo(movie, genre)
			


            if movie_index < upper_limit:
                movie_index += 50
            else:
                break

In [ ]:
def getUrl(_movie_count, _genre):
    if (_genre == 'documentary'):
        return ('https://www.imdb.com/search/title/?genres=documentary&start=' + str(_movie_count) + 
                '&explore=genres&ref_=adv_nxt')
    else:
        return ('https://www.imdb.com/search/title/?title_type=feature&genres=' + _genre + '&start=' + 
                str(_movie_count) + '&explore=genres&ref_=adv_nxt')


In [ ]:
# find and calculate the number of pages to iterate through based on the number of films in the list
def setMovieCountTotal(_g):
    url_temp = getUrl(1, _g)
    page = requests.get(url_temp)
    soup = BeautifulSoup(page.content, 'html.parser')
    temp = soup.find(class_='desc')

    temp = temp.span.get_text().split()[2]
    temp = temp.replace(',', '')
    print ('total movies: ' + temp)
    return int(temp)
	

In [ ]:
def stripInfo(_movie, _genre):

    movie_id = _movie.find(class_ = 'ribbonize').get('data-tconst')
	
    title = _movie.find(class_ = 'lister-item-header')
    if not(title == None):
        title = title.a.get_text()

    classification = _movie.find(class_='certificate')
    if not(classification == None):
        classification = classification.get_text()

    runtime = _movie.find(class_='runtime')
    if not(runtime == None):
        runtime = runtime.get_text().split()[0]

    year = _movie.find(class_='lister-item-year text-muted unbold')
    if not(year == None):
        year = year.get_text()[-5:-1]

        # make sure the text gather is in the YYYY format, otherwise set to None
        if not(re.search('[0-9][0-9][0-9][0-9]', year)):
            year = None

    rating = _movie.find(class_='inline-block ratings-imdb-rating')
    if not(rating == None):
        rating = rating.get_text().strip()

    metascore = _movie.find(class_='inline-block ratings-metascore')
    if not(metascore == None):
        metascore = metascore.get_text().split()[0]
    
    data = {"ID": movie_id,
			"movie_title": title,
            "class": classification, 
            "year": year,
            "runtime": runtime ,
            "rating": rating,
            "meta": metascore}
    imdb.append(data)	
	

In [ ]:
extract()

total movies: 47287
Genre:		action
Num Pages:	945

total movies: 23238
Genre:		adventure
Num Pages:	464

total movies: 6887
Genre:		animation
Num Pages:	137

total movies: 6962
Genre:		biography
Num Pages:	139

total movies: 94018
Genre:		comedy
Num Pages:	1880

total movies: 31575
Genre:		crime
Num Pages:	631

total movies: 9151
Genre:		war
Num Pages:	183

total movies: 196505
Genre:		drama
Num Pages:	3930

total movies: 14946
Genre:		family
Num Pages:	298

total movies: 14615
Genre:		fantasy
Num Pages:	292

total movies: 7815
Genre:		history
Num Pages:	156

total movies: 30705
Genre:		horror
Num Pages:	614

total movies: 6564
Genre:		music
Num Pages:	131

total movies: 9817
Genre:		musical
Num Pages:	196

total movies: 15949
Genre:		mystery
Num Pages:	318

total movies: 46365
Genre:		romance
Num Pages:	927

total movies: 14167
Genre:		sci-fi
Num Pages:	283

total movies: 4543
Genre:		sport
Num Pages:	90

total movies: 45471
Genre:		thriller
Num Pages:	909

total movies: 8632
Genre:		

In [ ]:
df = pd.DataFrame(imdb)

In [ ]:
df.shape

(634750, 7)

In [ ]:
df.to_csv('data_movie.csv',index=False)

In [ ]:
from google.colab import files
files.download('data_movie.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>